<a href="https://colab.research.google.com/github/Adlucent/ga4-churn-prediction/blob/main/V_Aggregation_and_Initial_Feature_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# V. Aggregation by CustID and Initial Feature Selection
In this notebook, we'll review all predictors one by one to perform final cleaning and then aggregation at the customer ID level.

# Import libraries

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

project_name = 'adl-analytics' #add proj name
region = "US"  # GCP project region

In [ ]:
# Google credentials 
from google.colab import auth
auth.authenticate_user()

# BigQuery Magics 
'''BigQuery magics are used to run BigQuery SQL queries in a python environment.
These queries can also be run in the BigQuery UI '''

from google.cloud import bigquery
from google.cloud.bigquery import magics, Client, QueryJobConfig

magics.context.project = project_name
client = bigquery.Client(project=magics.context.project)

# data processing
import pandas as pd
import numpy as np
from datetime import date, timedelta, timezone
import re #regex library

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# suppress notebook warnings
import warnings
warnings.filterwarnings('ignore')

# dataframe formatting
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", lambda x: "%.3f" % x)

## Load BigQuery table saved at the end of the third (not fourth) notebook

In this notebook, we'll pick up where we left off in notebook three in prepairing the training data. Notebook four is where we created the churn target, which we'll append to the training set in a later step.

In [ ]:
%%bigquery data --project $project_name
SELECT *
FROM `adl-analytics.public_sample_data.ga4_step_3_colab` # update with your project and dataset name

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
# make a copy to retain original import
df = data.copy()

In [ ]:
# spot check first five rows
df.head()

,pk,event_date,event_timestamp,event_name,event_value_in_usd,user_pseudo_id,user_first_touch_timestamp,user_ltv_revenue,device_category,device_mobile_brand_name,device_mobile_model_name,device_operating_system,device_operating_system_version,device_language,device_web_info_browser,device_web_info_browser_version,geo_continent,geo_sub_continent,geo_country,traffic_source_medium,traffic_source_name,traffic_source_source,ecommerce_total_item_quantity,ecommerce_purchase_revenue_in_usd,ecommerce_purchase_revenue,ecommerce_tax_value_in_usd,ecommerce_tax_value,ecommerce_unique_items,ecommerce_transaction_id,foreign_key,campaign,coupon,engagement_time_msec,ga_session_id,ga_session_number,link_url,medium,promotion_name,session_engaged,tax,transaction_id,value,items_item_brand,items_item_variant,items_item_category,items_price_in_usd,items_price,items_quantity,items_item_revenue_in_usd,items_item_revenue,items_promotion_name,items_creative_name,shipping_tier_provider,shipping_tier_type
0,1607461238808403|begin_checkout,2020-12-08,1607461238808403,begin_checkout,NaN,52407464.0313773688,1605035923000488,0.000,mobile,Huawei,Other,Web,10,None,Chrome,87_0,Asia,Western_Asia,Qatar,referral,referral,shop_googlemerchandisestore_com,1,NaN,NaN,NaN,NaN,1,not_set,1607461238808403|begin_checkout,None,NaN,0.000,118606764.0,5.000,0,None,None,1.000,0.000,NaN,0.000,Google,MD,unknown,NaN,30.000,1,NaN,NaN,not_set,not_set,None,Rush
1,1611433281755792|add_to_cart,2021-01-23,1611433281755792,add_to_cart,NaN,53469191.6312753520,1604943514229692,145.000,mobile,Samsung,Other,Android,Other,None,Other,Other,Africa,Western_Africa,Nigeria,referral,referral,shop_googlemerchandisestore_com,1,NaN,NaN,NaN,NaN,12,not_set,1611433281755792|add_to_cart,None,NaN,9.000,45936623.0,10.000,0,None,None,1.000,0.000,NaN,0.000,Google,XL,Apparel,NaN,60.000,1,NaN,NaN,Act_Responsible,not_set,None,Rush
2,1606849666257790|begin_checkout,2020-12-01,1606849666257790,begin_checkout,NaN,5502112.0184769802,1606531073262181,0.000,mobile,Samsung,Other,Web,10,en,Other,Other,Europe,Eastern_Europe,Czechia,data_deleted,data_deleted,data_deleted,3,NaN,NaN,NaN,NaN,3,not_set,1606849666257790|begin_checkout,None,NaN,1.000,1221594038.0,5.000,0,None,None,1.000,0.000,NaN,0.000,Android,Toddler,Apparel,NaN,18.000,1,NaN,NaN,none,not_set,None,Rush
3,1606758653555162|begin_checkout,2020-11-30,1606758653555162,begin_checkout,NaN,5502112.0184769802,1606531073262181,0.000,mobile,Samsung,Other,Web,10,en,Other,Other,Europe,Eastern_Europe,Czechia,referral,referral,shop_googlemerchandisestore_com,2,NaN,NaN,NaN,NaN,2,not_set,1606758653555162|begin_checkout,None,NaN,273.000,790620521.0,3.000,0,None,None,1.000,0.000,NaN,0.000,Android,Toddler,Apparel,NaN,18.000,1,NaN,NaN,Reach_New_Heights,not_set,None,Rush
4,1606758742620311|begin_checkout,2020-11-30,1606758742620311,begin_checkout,NaN,5502112.0184769802,1606531073262181,0.000,mobile,Samsung,Other,Web,10,en,Other,Other,Europe,Eastern_Europe,Czechia,referral,referral,shop_googlemerchandisestore_com,2,NaN,NaN,NaN,NaN,2,not_set,1606758742620311|begin_checkout,None,NaN,1.000,790620521.0,3.000,0,None,None,1.000,0.000,NaN,0.000,Android,Toddler,Apparel,NaN,18.000,1,NaN,NaN,Not_available_in_demo_dataset,not_set,None,Rush


First, remove post-cutoff data to avoid data leakage.

In [ ]:
# set Cutoff Date at 28 days prior to the end of the table date range (as done in prior steps)
df["event_date"] = pd.to_datetime(df["event_date"]) # convert to datetime in pandas before proceeding
cutoff_date = df["event_date"].max() - timedelta(days=28)
print(cutoff_date)

2021-01-03 00:00:00


In [ ]:
# separate pre and post cutoff tables
df_pre = pd.DataFrame(df.loc[df["event_date"] < cutoff_date])
df_post = pd.DataFrame(df.loc[df["event_date"] >= cutoff_date])

From this point forward, we'll look at only the pre-cutoff table.

### Create a table containing total number of each event type per user. We'll do this by isolating user id, event name and the private key into a table, then using the crosstab function to pivot and count the values. 

In [ ]:
df1 = df_pre[
    [
        "user_pseudo_id",
        "event_name",
        "pk",
    ]
].copy()

df1.drop_duplicates(subset=None, keep="first", inplace=True)

In [ ]:
df_agg = pd.crosstab(index = df1['user_pseudo_id'], columns = df1['event_name']).rename_axis(columns=None)

In [ ]:
# dynamically rename columns, inserting eventCount_ in front of each new column name
x = df_agg.columns.tolist()
col = "eventCount"
df_agg = df_agg.rename(columns=lambda x: str(col) + '_' + str(x)).reset_index()
df_agg.drop_duplicates(subset=None, keep="first", inplace=True)

In [ ]:
# check to make sure all looks as expected
df_agg.head()

,user_pseudo_id,eventCount_add_payment_info,eventCount_add_shipping_info,eventCount_add_to_cart,eventCount_begin_checkout,eventCount_click,eventCount_first_visit,eventCount_page_view,eventCount_purchase,eventCount_scroll,eventCount_select_item,eventCount_select_promotion,eventCount_session_start,eventCount_user_engagement,eventCount_view_item,eventCount_view_promotion,eventCount_view_search_results
0,10111055.8768683862,2,1,4,2,0,1,26,1,16,1,1,1,25,7,2,0
1,1014825.0200289249,2,2,0,2,0,1,17,1,11,0,0,2,17,6,1,0
2,1016446.8237887674,4,4,6,12,0,1,42,1,23,2,0,3,38,20,1,0
3,10172849.5375293351,2,1,7,2,0,1,61,1,32,6,0,3,55,21,5,0
4,1019527.5799124267,1,1,7,2,0,1,34,1,21,0,0,1,33,23,2,0


Now that we have our base table, let's go through the categorical features in sections.

### `Device` columns

In [ ]:
# create a list of the device-specific columns
device_cols = (
    "device_category",
    "device_mobile_brand_name",
    "device_mobile_model_name",
    "device_operating_system",
    "device_operating_system_version",
    "device_language",
    "device_web_info_browser",
    "device_web_info_browser_version"
)

# create df subset 
df2 = df_pre[[
    "user_pseudo_id", 
    "pk", 
    "device_category",
    "device_mobile_brand_name",
    "device_mobile_model_name",
    "device_operating_system",
    "device_operating_system_version",
    "device_language",
    "device_web_info_browser",
    "device_web_info_browser_version"
]].copy()

# drop duplicate rows
df2.drop_duplicates(subset=None, keep="first", inplace=True)

# print counts of unique values in each column
for col in device_cols:
    print(col)
    print(df2[col].value_counts())
    print("*" * 30)

device_category
desktop    356889
mobile     246200
tablet      13089
Name: device_category, dtype: int64
******************************
device_mobile_brand_name
Apple        261576
Google       221536
Samsung       47130
Other         45083
Microsoft     13306
Xiaomi        11526
Huawei         8400
Mozilla        7621
Name: device_mobile_brand_name, dtype: int64
******************************
device_mobile_model_name
Chrome        177519
iPhone        132256
Other         124975
Safari        120017
ChromeBook     32848
Edge           11592
iPad            7635
Firefox         7621
Pixel           1715
Name: device_mobile_model_name, dtype: int64
******************************
device_operating_system
Web          368975
iOS           70312
Windows       69447
Android       48217
Macintosh     42522
Other         16705
Name: device_operating_system, dtype: int64
******************************
device_operating_system_version
Other    233533
10       228286
10.15     80625
14.3      270

In [ ]:
# combine device_mobile_brand_name and device_mobile_model_name into one column, then drop original columns
df2['device_mobile_brand_model'] = df2['device_mobile_brand_name'] + '_' + df2['device_mobile_model_name']

df2.drop([
    'device_mobile_brand_name', 
    'device_mobile_model_name', 
], axis=1, inplace=True)

In [ ]:
# combine device_operating_system and device_operating_system_version into one column, then drop original columns
df2['device_os_and_version'] = df2['device_operating_system'] + '_' + df2['device_operating_system_version']

df2.drop([
    'device_operating_system', 
    'device_operating_system_version', 
], axis=1, inplace=True)

# clean up special characters
df2['device_os_and_version'] = df2['device_os_and_version'].str.replace('.', '_', regex = True)

In [ ]:
# combine device_operating_system and device_operating_system_version into one column, then drop original columns
df2['device_web_browser_and_version'] = df2['device_web_info_browser'] + '_' + df2['device_web_info_browser_version']

df2.drop([
    'device_web_info_browser', 
    'device_web_info_browser_version', 
], axis=1, inplace=True)

In [ ]:
# using the crosstab function again, append counts of each device column value onto our df_agg table
custid = "user_pseudo_id"
cols = (
    "device_category",
    "device_language",
    "device_mobile_brand_model",
    "device_os_and_version",
    "device_web_browser_and_version"
)

for col in cols:
    df_ctab = pd.crosstab(df2[custid], df2[col]).rename_axis(columns=None)
    x = df_ctab.columns.tolist()
    df_ctab = df_ctab.rename(columns=lambda x: str(col) + '_' + str(x)).reset_index()
    df_agg = pd.merge(df_agg,df_ctab, how='left', on = custid)

### `Geo` columns
We'll perform the same steps as above on geo columns, as well as several other column groups below.

In [ ]:
# isolate geo columns
geo_cols = (
    "geo_continent",
    "geo_sub_continent",
    "geo_country"
)

# create df subset
df3 = df_pre[[
    "user_pseudo_id", 
    "pk", 
    "geo_continent",
    "geo_sub_continent",
    "geo_country"
]].copy()

df3.drop_duplicates(subset=None, keep="first", inplace=True)

for col in geo_cols:
    print(col)
    print(df3[col].value_counts())
    print("*" * 30)

geo_continent
Americas    349451
Asia        141140
Europe      110939
Oceania       7045
Africa        5726
not_set       1877
Name: geo_continent, dtype: int64
******************************
geo_sub_continent
Northern_America    322838
Southern_Asia        64158
Eastern_Asia         37636
Western_Europe       35886
Northern_Europe      30230
Southern_Europe      26647
Southeast_Asia       22650
Eastern_Europe       18176
South_America        17820
Western_Asia         15993
Australasia           7045
Central_America       6804
Northern_Africa       3106
Caribbean             1989
not_set               1877
Western_Africa        1617
Eastern_Africa         710
Central_Asia           703
Southern_Africa        293
Name: geo_sub_continent, dtype: int64
******************************
geo_country
United_States             274926
India                      59365
Canada                     47912
United_Kingdom             17747
France                     11584
Spain                      111

In [ ]:
# drop geo_continent and geo_country due to the large number of unique values, keeping geo_sub_continent to represent geographic information
df3.drop([
    'geo_continent', 
    'geo_country', 
], axis=1, inplace=True)

In [ ]:
# append counts of each geo_sub_continent value onto the agg table
df_ctab = pd.crosstab(df3["user_pseudo_id"], df3["geo_sub_continent"]).rename_axis(columns=None)
x = df_ctab.columns.tolist()
df_ctab = df_ctab.rename(columns=lambda x: str("geo_sub_continent") + '_' + str(x)).reset_index()
df_agg = pd.merge(df_agg,df_ctab, how='left', on = "user_pseudo_id")
df_agg.drop_duplicates(subset=None, keep="first", inplace=True)

### `Ecommerce` columns
Same steps here as above.

In [ ]:
# isolate ecom columns
ecom_cols = (
    "ecommerce_purchase_revenue",
    "ecommerce_purchase_revenue_in_usd",
    "ecommerce_tax_value",
    "ecommerce_tax_value_in_usd",
    "ecommerce_total_item_quantity",
    "ecommerce_transaction_id",
    "ecommerce_unique_items"
)

# create df subset
df4 = df_pre[[
    "user_pseudo_id", 
    "pk", 
    "ecommerce_purchase_revenue",
    "ecommerce_purchase_revenue_in_usd",
    "ecommerce_tax_value",
    "ecommerce_tax_value_in_usd",
    "ecommerce_total_item_quantity",
    "ecommerce_transaction_id",
    "ecommerce_unique_items"
]].copy()

df4.drop_duplicates(subset=None, keep="first", inplace=True)

for col in ecom_cols:
    print(col)
    print(df4.dtypes[col])
    print(df4[col].nunique())
    print("*" * 30)

ecommerce_purchase_revenue
float64
329
******************************
ecommerce_purchase_revenue_in_usd
float64
330
******************************
ecommerce_tax_value
float64
54
******************************
ecommerce_tax_value_in_usd
float64
54
******************************
ecommerce_total_item_quantity
Int64
73
******************************
ecommerce_transaction_id
object
3589
******************************
ecommerce_unique_items
Int64
27
******************************


In [ ]:
# drop ecommerce_purchase_revenue and ecommerce_tax_value, and keep the _in_usd versions of each
# using the usd versions of currency columns will ensure consistency
df4.drop([
    'ecommerce_purchase_revenue', 
    'ecommerce_tax_value', 
], axis=1, inplace=True)

In [ ]:
# add the values of the aggregated cols at the cust id level and append to df_agg
df4 = df4.groupby(["user_pseudo_id"]).agg(
        sum_ecom_purchase_revenue = ("ecommerce_purchase_revenue_in_usd", "sum"),
        sum_ecom_tax_value = ("ecommerce_tax_value_in_usd", "sum"),
        sum_ecom_total_item_qty = ("ecommerce_total_item_quantity", "sum"),
        max_ecom_unique_items = ("ecommerce_unique_items", "max"),
)
df4.reset_index(inplace=True)
df_agg = pd.merge(df_agg, df4, how='left', on = "user_pseudo_id")

### `Item` columns

In [ ]:
# isolate item columns
item_cols = (
    "items_creative_name",
    "items_item_brand",
    "items_item_category",
    "items_item_revenue",
    "items_item_revenue_in_usd",
    "items_price",
    "items_price_in_usd",
    "items_item_variant",
    "items_promotion_name",
    "items_quantity"
)

# create df subset
df5 = df_pre[[
    "user_pseudo_id", 
    "pk", 
    "items_creative_name",
    "items_item_brand",
    "items_item_category",
    "items_item_revenue",
    "items_item_revenue_in_usd",
    "items_price",
    "items_price_in_usd",
    "items_item_variant",
    "items_promotion_name",
    "items_quantity"
]].copy()

df5.drop_duplicates(subset=None, keep="first", inplace=True)

for col in item_cols:
    print(col)
    print(df5.dtypes[col])
    print(df5[col].nunique())
    print("*" * 30)

items_creative_name
object
2
******************************
items_item_brand
object
7
******************************
items_item_category
object
18
******************************
items_item_revenue
float64
164
******************************
items_item_revenue_in_usd
float64
164
******************************
items_price
float64
72
******************************
items_price_in_usd
float64
66
******************************
items_item_variant
object
16
******************************
items_promotion_name
object
7
******************************
items_quantity
Int64
45
******************************


In [ ]:
# drop items_item_revenue and items_price and keep the _in_usd versions of each
df5.drop([
    'items_item_revenue', 
    'items_price', 
], axis=1, inplace=True)

In [ ]:
# aggregate numeric cols from items
df5_num = df5.groupby(["user_pseudo_id"]).agg(
        sum_items_revenue = ("items_item_revenue_in_usd", "sum"),
        sum_items_price = ("items_price_in_usd", "sum"),
        sum_items_quantity = ("items_quantity", "sum")
)
df5_num.reset_index(inplace=True)
df_agg = pd.merge(df_agg, df5_num, how='left', on = "user_pseudo_id")

In [ ]:
# aggregate categorical cols from items
custid = "user_pseudo_id"
cols = (
    "items_creative_name",
    "items_item_brand",
    "items_item_category",
    "items_promotion_name",
    "items_item_variant"
)

for col in cols:
    df_ctab = pd.crosstab(df5[custid], df5[col]).rename_axis(columns=None)
    x = df_ctab.columns.tolist()
    df_ctab = df_ctab.rename(columns=lambda x: str(col) + '_' + str(x)).reset_index()
    df_agg = pd.merge(df_agg,df_ctab, how='left', on = custid)

### Remaining event-specific fields

In [ ]:
custid = "user_pseudo_id"
cols = (
    "medium",
    "shipping_tier_provider",
    "shipping_tier_type",
    "session_engaged"
)

df6 = df_pre[[
    "user_pseudo_id", 
    "pk", 
    "medium",
    "shipping_tier_provider",
    "shipping_tier_type",
    "session_engaged",
    "event_value_in_usd",
    "value",
    "engagement_time_msec"
]].copy()

df6.drop_duplicates(subset=None, keep="first", inplace=True)

# categorical cols
for col in cols:
    df_ctab = pd.crosstab(df6[custid], df6[col]).rename_axis(columns=None)
    x = df_ctab.columns.tolist()
    df_ctab = df_ctab.rename(columns=lambda x: str(col) + '_' + str(x)).reset_index()
    df_agg = pd.merge(df_agg,df_ctab, how='left', on = custid)

# numeric cols
df6_agg = df6.groupby(custid).agg(
    sum_event_value = ("event_value_in_usd", "sum"),
    sum_value = ("value", "sum"),
    sum_engagement_time = ("engagement_time_msec", "sum")
)
df6_agg.reset_index(inplace=True)
df_agg = pd.merge(df_agg, df6_agg, how='left', on = custid)

## User-specific fields

For these fields, since they tell us information about each customer that does not change (or doesn't change often), instead of aggregating them, we want to know what the most recent value for each is.

In [ ]:
# categories
user_cat_cols = (
    "traffic_source_medium",
    "traffic_source_name",
    "traffic_source_source",
)

# numeric cols
user_num_cols = (
    "user_first_touch_timestamp",
    "user_ltv_revenue",
)

df_user = df_pre[[
    "user_pseudo_id", 
    "event_timestamp",
    "pk", 
    "traffic_source_medium",
    "traffic_source_name",
    "traffic_source_source",
    "user_first_touch_timestamp",
    "user_ltv_revenue"
]].copy()

df_user.drop_duplicates(subset=None, keep="first", inplace=True)

In [ ]:
# combine traffic_source_medium and traffic_source_source and then drop both original cols; this will catch the segments within both without losing any information
df_user['user_acquisition_channel'] = df_user['traffic_source_medium'] + '_' + df_user['traffic_source_source']

# drop traffic_source_name and original cols from above. traffic_source_name does not provide enough additional info to keep it
df_user.drop([
    'traffic_source_name', 
    'traffic_source_medium',
    'traffic_source_source',
], axis=1, inplace=True)

In [ ]:
df_user.head()

,user_pseudo_id,event_timestamp,pk,user_first_touch_timestamp,user_ltv_revenue,user_acquisition_channel
0,52407464.0313773688,1607461238808403,1607461238808403|begin_checkout,1605035923000488,0.000,referral_shop_googlemerchandisestore_com
2,5502112.0184769802,1606849666257790,1606849666257790|begin_checkout,1606531073262181,0.000,data_deleted_data_deleted
3,5502112.0184769802,1606758653555162,1606758653555162|begin_checkout,1606531073262181,0.000,referral_shop_googlemerchandisestore_com
4,5502112.0184769802,1606758742620311,1606758742620311|begin_checkout,1606531073262181,0.000,referral_shop_googlemerchandisestore_com
5,15492110.1286918774,1604570014837499,1604570014837499|purchase,1604430845829674,66.000,none_direct


In [ ]:
# get the most recent record per user for user_aquisition_channel
df_user_1 = df_user[df_user.groupby('user_pseudo_id')['event_timestamp'].transform(lambda x: (x.max() == x))]
df_user_1.reset_index(inplace=True)
df_user_1 = df_user_1[[
    "user_pseudo_id",
    "user_acquisition_channel"
]]
df_user.drop("user_acquisition_channel", axis = 1, inplace = True) # drop from df_user to prevent duplicating columns

df_ctab = pd.crosstab(df_user_1["user_pseudo_id"], df_user_1["user_acquisition_channel"]).rename_axis(columns=None)
x = df_ctab.columns.tolist()
df_ctab = df_ctab.rename(columns=lambda x: str("user_acquisition_channel") + '_' + str(x)).reset_index()
df_agg = pd.merge(df_agg,df_ctab, how='left', on = custid)

In [ ]:
df_agg.head()

,user_pseudo_id,eventCount_add_payment_info,eventCount_add_shipping_info,eventCount_add_to_cart,eventCount_begin_checkout,eventCount_click,eventCount_first_visit,eventCount_page_view,eventCount_purchase,eventCount_scroll,eventCount_select_item,eventCount_select_promotion,eventCount_session_start,eventCount_user_engagement,eventCount_view_item,eventCount_view_promotion,eventCount_view_search_results,device_category_desktop,device_category_mobile,device_category_tablet,device_language_de,device_language_en,device_language_es,device_language_fr,device_language_ko,device_language_zh,device_mobile_brand_model_Apple_Other,device_mobile_brand_model_Apple_Safari,device_mobile_brand_model_Apple_iPad,device_mobile_brand_model_Apple_iPhone,device_mobile_brand_model_Google_Chrome,device_mobile_brand_model_Google_ChromeBook,device_mobile_brand_model_Google_Other,device_mobile_brand_model_Google_Pixel,device_mobile_brand_model_Huawei_Other,device_mobile_brand_model_Microsoft_Edge,device_mobile_brand_model_Microsoft_Other,device_mobile_brand_model_Mozilla_Firefox,device_mobile_brand_model_Other_Other,device_mobile_brand_model_Samsung_Other,device_mobile_brand_model_Xiaomi_Other,device_os_and_version_Android_10,device_os_and_version_Android_9,device_os_and_version_Android_Other,device_os_and_version_Macintosh_10_15,device_os_and_version_Macintosh_11_1,device_os_and_version_Macintosh_Other,device_os_and_version_Other_Other,device_os_and_version_Web_10,device_os_and_version_Web_10_15,device_os_and_version_Web_Other,device_os_and_version_Windows_10,device_os_and_version_Windows_7,device_os_and_version_Windows_Other,device_os_and_version_iOS_14_2,device_os_and_version_iOS_14_3,device_os_and_version_iOS_Other,device_web_browser_and_version_Android_Webview_86_0,device_web_browser_and_version_Android_Webview_87_0,device_web_browser_and_version_Android_Webview_Other,device_web_browser_and_version_Chrome_86_0,device_web_browser_and_version_Chrome_87_0,device_web_browser_and_version_Chrome_Other,device_web_browser_and_version_Edge_86_0,device_web_browser_and_version_Edge_87_0,device_web_browser_and_version_Edge_Other,device_web_browser_and_version_Firefox_82_0,device_web_browser_and_version_Firefox_83_0,device_web_browser_and_version_Firefox_84_0,device_web_browser_and_version_Firefox_Other,device_web_browser_and_version_Other_Other,device_web_browser_and_version_Safari_13_0,device_web_browser_and_version_Safari_13_1,device_web_browser_and_version_Safari_14_0,device_web_browser_and_version_Safari_604,device_web_browser_and_version_Safari_Other,geo_sub_continent_Australasia,geo_sub_continent_Caribbean,geo_sub_continent_Central_America,geo_sub_continent_Central_Asia,geo_sub_continent_Eastern_Africa,geo_sub_continent_Eastern_Asia,geo_sub_continent_Eastern_Europe,geo_sub_continent_Northern_Africa,geo_sub_continent_Northern_America,geo_sub_continent_Northern_Europe,geo_sub_continent_South_America,geo_sub_continent_Southeast_Asia,geo_sub_continent_Southern_Africa,geo_sub_continent_Southern_Asia,geo_sub_continent_Southern_Europe,geo_sub_continent_Western_Africa,geo_sub_continent_Western_Asia,geo_sub_continent_Western_Europe,geo_sub_continent_not_set,sum_ecom_purchase_revenue,sum_ecom_tax_value,sum_ecom_total_item_qty,max_ecom_unique_items,sum_items_revenue,sum_items_price,sum_items_quantity,items_creative_name_Front_Page_Carousel,items_creative_name_not_set,items_item_brand_Android,items_item_brand_Google,items_item_brand_Google_Cloud,items_item_brand_IamRemarkable,items_item_brand_YouTube,items_item_brand_none,items_item_brand_not_set,items_item_category_Accessories,items_item_category_Android,items_item_category_Apparel,items_item_category_Black_Lives_Matter,items_item_category_Campus_Collection,items_item_category_Eco,items_item_category_Gift_Cards,items_item_category_Google,items_item_category_IamRemarkable,items_item_category_Lifestyle,items_item_category_New,items_item_category_Other,items_item_category_Sale,items_item_category_Stationary,items_item_cat

In [ ]:
# check the df shape
df_agg.shape

(3617, 180)

In [ ]:
# get the most recent record per user of ltv revenue; exclude user first touch timestamp, as we'll account for that in the next step with a feature equaling max = min event timestamp
df_user_2 = df_user[df_user.groupby('user_pseudo_id')['event_timestamp'].transform(lambda x: (x.max() == x))]
df_user_2.reset_index(inplace=True)
df_user_2 = df_user_2[[
    "user_pseudo_id",
    "user_ltv_revenue"
]]

# calculate days since user_first_touch_timestamp by doing snapshot dat
df_agg = pd.merge(df_agg,df_user_2, how='left', on = custid)

### Final date fields

In [ ]:
df['event_timestamp'] = df['event_timestamp'].astype(float)

In [ ]:
# isolate unique rows of user id, private key, event date and event timestamp
df_dates = df_pre[[
    "user_pseudo_id", 
    "pk", 
    "event_date",
    "event_timestamp",
]].copy()

df_dates.drop_duplicates(subset=None, keep="first", inplace=True)

In [ ]:
# define snapshot date as the day after the cutoff date
snapshot_date = df["event_date"].max() + timedelta(days=1)

# group by user id and calculate recency (in number days), and range of event_timestamp (the amount of time between their first and last interaction as of the cutoff date)
df_dates = df_dates.groupby("user_pseudo_id").agg(
        event_date_recency = ("event_date", lambda x: (snapshot_date - x.max()).days),
        event_timestamp_range = ("event_timestamp", lambda x: (x.max() - x.min()))
)
df_dates.reset_index(inplace=True)

In [ ]:
# add to df_agg
df_agg = pd.merge(df_agg, df_dates, how='left', on = "user_pseudo_id")

Next, add Recency column to indicate last purchase date per user (this is different than the columns we added for last event date and date between first and last event, since that dataset contains more than just purchases).

To do this, we'll load User ID and Recency columns from table saved at the end of the RFM notebook.

In [ ]:
%%bigquery df_rfm --project $project_name 
SELECT user_pseudo_id, Recency 
FROM `adl-analytics.public_sample_data.ga4_rfm` # update with your project and dataset name

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df_rfm.head()

,user_pseudo_id,Recency
0,1494018.5183133777,63
1,2422026.0165722609,63
2,3297046.6509553424,63
3,33027284.2974994612,63
4,4075022.3227633472,63


In [ ]:
# add Recency to df_agg
df_agg = pd.merge(df_agg, df_rfm, how='left', on = "user_pseudo_id")

Append Churn Target by loading the BigQuery table we saved at the end of the last (fourth) notebook.

In [ ]:
%%bigquery df_target --project $project_name
SELECT *
FROM `adl-analytics.public_sample_data.ga4_step_4_colab` # update with your project and dataset name

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df_target.head()

,user_pseudo_id,Churn_Target
0,1056498.5939985013,0
1,11677998.3685150844,0
2,1279595.7348368743,0
3,14670836.5443476050,0
4,21242720.7094333523,0


In [ ]:
df_target["Churn_Target"].value_counts()

1    1540
0      30
Name: Churn_Target, dtype: Int64

In [ ]:
# add churn target column to df_agg
df_agg_target = pd.merge(df_target, df_agg, how='left', on = "user_pseudo_id")

In [ ]:
# drop duplicate rows
df_agg_target.drop_duplicates(subset=None, keep="first", inplace=True)

In [ ]:
# check data types
df_agg_target.dtypes

user_pseudo_id                                                        object
Churn_Target                                                           Int64
eventCount_add_payment_info                                            int64
eventCount_add_shipping_info                                           int64
eventCount_add_to_cart                                                 int64
eventCount_begin_checkout                                              int64
eventCount_click                                                       int64
eventCount_first_visit                                                 int64
eventCount_page_view                                                   int64
eventCount_purchase                                                    int64
eventCount_scroll                                                      int64
eventCount_select_item                                                 int64
eventCount_select_promotion                                            int64

In [ ]:
df_agg_target["Churn_Target"].value_counts()

1    1540
0      30
Name: Churn_Target, dtype: Int64

In [ ]:
# final cleanup of columns headers before saving to BQ
df_agg_target.columns = [re.sub("[ ,-]", "_", re.sub("[\.,`,\$]", "", c)) for c in df_agg_target.columns]

## Save final aggregated table to BigQuery

In [ ]:
client = bigquery.Client(project = project_name)
table_id = 'public_sample_data.ga4_step_5_colab' # update with your project and dataset name
    
job_config = bigquery.LoadJobConfig(
    write_disposition = bigquery.job.WriteDisposition.WRITE_TRUNCATE
)

job = client.load_table_from_dataframe(
    df_agg_target, table_id, job_config = job_config
)

# Wait for the load job to complete.
job.result()

LoadJob<project=adl-analytics, location=US, id=3d0b5542-8567-4459-ac04-b649d76a0ea2>